In [315]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Data Processing

In [316]:
path = os.getcwd()
os.listdir()

['.ipynb_checkpoints',
 '1_cleansing.ipynb',
 '2_Modelling.ipynb',
 'archive.zip',
 'flightdata.csv',
 'mixed_delays.csv',
 'mixed_flight_data.csv',
 'TransformedDelays.csv',
 'WeatherEvents.csv']

[Kaggle: Weather Data](https://www.kaggle.com/sobhanmoosavi/us-weather-events)

In [317]:
weather = pd.read_csv(path + '/WeatherEvents.csv')
delays = pd.read_csv(path + '/flightdata.csv')
delays.head()

,airport,avg_arr,avg_dep,carrier,datetime,lat,lon,med_arr,med_dep,num_obs
0,MDW,22.934498,34.672489,WN,2018-01-01,41.785833,-87.752500,14.0,28.0,229
1,LAS,9.213930,18.029851,WN,2018-01-01,36.080000,-115.152222,4.0,9.0,201
2,BWI,13.202830,18.396226,WN,2018-01-01,39.175833,-76.668889,7.5,11.0,212
3,DEN,11.213542,21.291667,WN,2018-01-01,39.861667,-104.673056,1.0,8.0,192
4,DAL,19.432584,26.544944,WN,2018-01-01,32.845833,-96.850833,7.0,15.5,178


In [318]:
delays = delays.groupby(['airport','datetime']).agg({'avg_dep': 'mean', 'num_obs': 'sum'}).reset_index()

In [319]:
weather.head()

,EventId,Type,Severity,StartTime(UTC),EndTime(UTC),TimeZone,AirportCode,LocationLat,LocationLng,City,County,State,ZipCode
0,W-1,Snow,Light,2016-01-06 23:14:00,2016-01-07 00:34:00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0
1,W-2,Snow,Light,2016-01-07 04:14:00,2016-01-07 04:54:00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0
2,W-3,Snow,Light,2016-01-07 05:54:00,2016-01-07 15:34:00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0
3,W-4,Snow,Light,2016-01-08 05:34:00,2016-01-08 05:54:00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0
4,W-5,Snow,Light,2016-01-08 13:54:00,2016-01-08 15:54:00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0


In [320]:
weather.rename(columns = {'StartTime(UTC)': 'datetime', 'LocationLat': 'lat', 'LocationLng': 'lon'}, inplace = True)
weather.loc[:,'AirportCode'] = weather['AirportCode'].apply(lambda x: x[1:])
weather.loc[:,'datetime'] = pd.to_datetime(weather['datetime']).dt.floor('d')
delays.loc[:, 'datetime'] = pd.to_datetime(delays['datetime'])

In [321]:
minT = weather['datetime'].min()
maxT = weather['datetime'].max()

subset = delays[(delays['datetime'] >= minT) & (delays['datetime'] <= maxT)]

In [322]:
cols = ['avg_dep', 'airport', 'num_obs', 'datetime', 'Type', 'Severity']
df = subset.merge(weather, how = 'outer', left_on = ['airport', 'datetime'], right_on = ['AirportCode', 'datetime']).loc[:, cols]
df.loc[:,['Type','Severity']] = df[['Type','Severity']].fillna('Clear')
df.dropna(inplace = True)

In [323]:
df['weather'] = df['Type'] + ' ' + df['Severity']
df.drop(columns = ['Type', 'Severity'], inplace = True)
df.loc[:,'weather'] = df['weather'].replace('Clear Clear', 'Clear')

In [324]:
df.head()

,avg_dep,airport,num_obs,datetime,weather
0,7.500000,ABE,2.0,2018-01-01,Precipitation UNK
1,7.500000,ABE,2.0,2018-01-01,Precipitation UNK
2,17.500000,ABE,2.0,2018-01-02,Clear
3,6.333333,ABE,3.0,2018-01-03,Cold Severe
4,109.500000,ABE,2.0,2018-01-04,Snow Light


In [325]:
# There are multiple weather reports for any given day, I take the category that occurs the most 
# in order to reflect the fact that I am taking the mean departure delay per day
group = df.groupby(['airport', 'datetime', 'weather'])['avg_dep'].count()

rows = []
for i in group.index:
    x = group.loc[i[:2]].sort_values(ascending = False).index[0]
    rows.append(list(i) + [x])
    
new = pd.DataFrame(rows, columns = ['airport', 'datetime', 'weather', 'avg_weather']).drop('weather', axis = 1)

In [326]:
clean_df = df.merge(new, on = ['airport', 'datetime']).drop('weather', axis = 1).drop_duplicates()

In [327]:
clean_df.to_csv('mixed_delays.csv', index = False)